In [1]:
import dolfin as dl
import matplotlib.pyplot as plt
import numpy as np

from spin.core import problem
from spin.fenics import converter as fex_converter
from spin.hippylib import prior

In [47]:
# mesh = dl.IntervalMesh(100, -1, 1)
mesh = dl.RectangleMesh(dl.Point(-1, -1), dl.Point(1, 1), 10, 10)
problem_settings = problem.SPINProblemSettings(
    mesh=mesh,
    pde_type="mean_exit_time_moments",
    inference_type="drift_only",
    drift=(("-x[0]", "-x[1]")),
    log_squared_diffusion=("std::log(1.0)", "std::log(1.0)"),
)

In [48]:
problem_builder = problem.SPINProblemBuilder(problem_settings)
spin_problem = problem_builder.build()

In [4]:
prior_settings = prior.PriorSettings(
    function_space = spin_problem.function_space_parameters,
    mean=("-0.5*x[0]", "-0.5*x[1]"),
    variance=("0.1", "0.1"),
    correlation_length=("0.1", "0.1"),
)
prior_builder = prior.BilaplacianVectorPriorBuilder(prior_settings)
spin_prior = prior_builder.build()

In [49]:
forward_vector = spin_problem.hippylib_variational_problem.generate_state()
adjoint_vector = spin_problem.hippylib_variational_problem.generate_state()
parameter_vector = spin_problem.hippylib_variational_problem.generate_parameter()
parameter_vector.set_local(-np.linspace(-1, 1, parameter_vector.size()))
state_list = [forward_vector, parameter_vector, adjoint_vector]
spin_problem.hippylib_variational_problem.solveFwd(forward_vector, state_list)

In [59]:
solution_array = fex_converter.convert_to_numpy(forward_vector, num_components=2)
spin_problem.function_space_variables.split()

[FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 153), FiniteElement('Lagrange', triangle, 1)),
 FunctionSpace(Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 153), FiniteElement('Lagrange', triangle, 1))]

In [ ]:
precision_array = np.linspace(0, 1, 50)
precision_vector = dl.Vector()
precision_vector.init(precision_array.size)
precision_vector.set_local(precision_array)
precision_vector.apply("insert")


In [ ]:
observations = np.linspace(0, 1, 50)
obs_vec = dl.Vector(dl.MPI.comm_world)
obs_vec.size
test_matrix = dl.PETScMatrix(dl.MPI.comm_world)
test_matrix.set_diagonal(observations)

In [ ]:
from petsc4py import PETSc

observations = np.linspace(0, 1, 50)
petsc_matrix = PETSc.Mat().createAIJ(
        size=(observations.size, observations.size), comm=dl.MPI.comm_world
    )
petsc_matrix.setUp()
for i, value in enumerate(observations):
    petsc_matrix.setValues(i, i, value)
petsc_matrix.assemble()
precision_matrix = dl.PETScMatrix(petsc_matrix, comm=dl.MPI.comm_world)